<a href="https://colab.research.google.com/github/ViniciusARZ/Data-Structures-and-Algorithms/blob/main/Oneshot%20Analysis/Lost_Reasons_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Gspread Usual Stack
!pip install -q --upgrade gspread
!pip install -q pandas
!pip install -q -U -q PyDrive
!pip install -q datetime
!pip install -q pendulum
!pip install -q matplotlib


import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
import datetime
import pendulum

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 51 kB 138 kB/s 
     |████████████████████████████████| 251 kB 20.2 MB/s 
     |████████████████████████████████| 155 kB 5.7 MB/s 
     |████████████████████████████████| 489 kB 15.4 MB/s 


Stage History was extracted from Hubspot going to the property and exporting all of its historical data

In [ ]:
## Stage History
sh = pd.read_csv('/content/deal-stage.csv',on_bad_lines='skip')

losts = sh[(sh['Deal Stage Current Value']=='Lost') & (sh['Deal Stage Previous Value (1)'].isin(['Lead accepted (MQL)','Demo scheduled','Opportunity','SQL/Follow-up after demo','No show','Proposal sent - Negotiation','Lost','Offer accepted']))]
losts.head(3)

,Deal ID,Deal Name,Deal Stage Current Value,Deal Stage Change Date,Deal Stage Previous Value (1),Deal Stage Change Date (1),Deal Stage Previous Value (2),Deal Stage Change Date (2),...
4,1523946131,MX: Ah Cacao,Lost,2020-02-12 14:30,Demo scheduled,2020-02-06 16:56,NaN,NaN,NaN
5,1653701014,CO: COLORQUIMICA,Lost,2022-01-18 14:20,Lead accepted (MQL),2020-02-28 14:46,NaN,NaN,NaN
6,4877856301,CO: Cafasur Renewal,Lost,2021-06-28 20:52,Opportunity,2021-04-12 20:02,SQL/Follow-up after demo,2021-03-30 17:10,NaN


In [ ]:
losts.columns

Index(['Deal ID', 'Deal Name', 'Deal Stage Current Value',
       'Deal Stage Change Date', 'Deal Stage Previous Value (1)',
       'Deal Stage Change Date (1)', 'Deal Stage Previous Value (2)',
       'Deal Stage Change Date (2)', '...'],
      dtype='object')

In [ ]:
import plotly.express as px
losts['Deal Stage Change Date'] = pd.to_datetime(losts['Deal Stage Change Date'])
group_previous_stage_month = losts[['Deal ID','Deal Stage Change Date','Deal Stage Previous Value (1)']].groupby([pd.Grouper(key='Deal Stage Change Date',freq='M'),'Deal Stage Previous Value (1)']).nunique()
group_previous_stage_month = group_previous_stage_month.groupby(level=0).apply(lambda x: x / float(x.sum())).reset_index()
group_previous_stage_month.style.format({'Deal ID': "{:.2%}"})

fig = px.bar(group_previous_stage_month, x='Deal Stage Change Date', y='Deal ID',text_auto='.1%',color="Deal Stage Previous Value (1)")
fig.update_layout(yaxis={"dtick":1},margin={"t":70,"b":0},height=700,title_text='Lost Deals by Stage through time')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Salesforce Losts

In [ ]:
!pip install -q simple_salesforce
# installing salesforce access and giving auth
import requests
consumer_key = "3MVG9p1Q1BCe9GmCjBcLl1WrzgMV_tAn1ES_O4voJ3S_nbtljEik0gqbgz7Im43Y5CmxgeBHU3GOPAeA_Or7u"
consumer_secret = "BB065F99EDDDFCC55B0AE8A65786FC186513F0BEB07C7FFE0E13E13A4A01060F"
redirect_url = "http://localhost/"
sf_user = "v.ramos@slangapp.com"
sf_pass = "Sawarineko17*"
auth_url = "https://login.salesforce.com/services/oauth2/token"

response = requests.post(auth_url, data= {'client_id':consumer_key,
                                          'client_secret':consumer_secret,
                                          'grant_type':'password',
                                          'username':sf_user,
                                          'password':sf_pass})
json_res = response.json()
access_token = json_res['access_token']
auth = {'Authorization':'Bearer ' + access_token}
instance_url = json_res['instance_url']

url = instance_url + '/services/data/v45.0/sobjects/contact/describe'
res = requests.get(url,headers=auth)
r = res.json()

# Accessing the API and creating a SOQL Function to transform json results into dataframe
from simple_salesforce import Salesforce
isSandbox = 0
sec_token = 'cxy8s9babgRJNODYHyZqJXP4u'
sf = Salesforce(username = sf_user,
                password = sf_pass,
                security_token = sec_token,
                instance_url = instance_url)

def SOQL(SOQL):
  qResult = sf.query(SOQL)
  print('Record Count {0}'.format(qResult['totalSize']))
  isDone = qResult['done'];
  if isDone == True:
    df = pd.DataFrame(qResult['records'])

  while isDone != True:
    try:
        if qResult['done'] != True:
          df = df.append(pd.DataFrame(qResult['records']));
          qResult = sf.query_more(qResult['nextRecordsUrl'], True)
        else:
          df = df.append(pd.DataFrame(qResult['records']))
          isDone = True;
          print('completed')
          break;
    except NameError:
      df = pd.DataFrame(qResult['records'])
      qry = sf.query_more(qResult['nextRecordsUrl'], True)
      
  return df.drop('attributes',axis=1);


     |████████████████████████████████| 118 kB 7.8 MB/s 
     |████████████████████████████████| 100 kB 5.3 MB/s 
     |████████████████████████████████| 214 kB 45.4 MB/s 
     |████████████████████████████████| 4.0 MB 45.6 MB/s 
     |████████████████████████████████| 41 kB 643 kB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 6.4 MB 31.7 MB/s 


In [ ]:
sf_losts = SOQL("SELECT Hubspot_Deal_ID__c, Hubspot_Lost_Reaons_del__c, Name, StageName FROM Opportunity WHERE StageName = 'Closed Lost' AND Hubspot_Deal_ID__c	!= NULL")
sf_losts.head(2)

Record Count 12936
completed


,Hubspot_Deal_ID__c,Hubspot_Lost_Reaons_del__c,Name,StageName
0,2293896155,None,eMotiv: bmestas@prodigy.net.mx,Closed Lost
1,1782651536,None,EFlip: misslilich@gmail.com,Closed Lost


In [ ]:
losts['Deal ID'] = losts['Deal ID'].astype(str)
losts_total = sf_losts.merge(losts,left_on='Hubspot_Deal_ID__c',right_on='Deal ID',how='left')
losts_total.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Hubspot_Deal_ID__c,Hubspot_Lost_Reaons_del__c,Name,StageName,Deal ID,Deal Name,Deal Stage Current Value,Deal Stage Change Date,Deal Stage Previous Value (1),Deal Stage Change Date (1),Deal Stage Previous Value (2),Deal Stage Change Date (2),...
0,2293896155,None,eMotiv: bmestas@prodigy.net.mx,Closed Lost,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,1782651536,None,EFlip: misslilich@gmail.com,Closed Lost,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,1771018911,Other,EBen: eramos@agrosavia.co,Closed Lost,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN




```
Individual
Authority
Budget
Interest
Duplicate
License Quantity
Product
Competition
Not Contacted
Employees paid individually
Data Quality
Trial
Timing
Other
Administrative
Project Cancelled
```



In [ ]:
def has_value(df,Column,Substring):
  return np.where(df[Column].str.contains(Substring), 1, 0)

losts_total['Individual'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Individual')
losts_total['Authority'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Authority')
losts_total['Budget'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Budget')
losts_total['Interest'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Interest')
losts_total['Duplicate'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Duplicate')
losts_total['License Quantity'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','License Quantity')
losts_total['Product'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Product')
losts_total['Competition'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Competition')
losts_total['Not Contacted'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Not Contacted')
losts_total['Employees paid individually'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Employees paid individually')
losts_total['Data Quality'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Data Quality')
losts_total['Trial'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Trial')
losts_total['Timing'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Timing')
losts_total['Other'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Other')
losts_total['Administrative'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Administrative')
losts_total['Project Cancelled'] = has_value(losts_total,'Hubspot_Lost_Reaons_del__c','Project Cancelled')


losts_stage_reason = losts_total[['Deal ID','StageName','Hubspot_Lost_Reaons_del__c','Deal Stage Previous Value (1)','Individual','Authority','Budget','Interest','Duplicate','License Quantity','Product','Competition','Not Contacted','Employees paid individually','Data Quality','Trial','Timing','Other','Administrative','Project Cancelled']]

In [ ]:
group_reason_stage = losts_stage_reason.groupby(['Deal Stage Previous Value (1)']).sum()
group_reason_stage
import seaborn as sns
group_reason_stage.style.background_gradient(cmap='YlOrRd',axis=None)

,Individual,Authority,Budget,Interest,Duplicate,License Quantity,Product,Competition,Not Contacted,Employees paid individually,Data Quality,Trial,Timing,Other,Administrative,Project Cancelled
Deal Stage Previous Value (1),,,,,,,,,,,,,,,,
Demo scheduled,11,16,53,90,1,0,14,17,0,37,2,1,14,36,0,0
Lead accepted (MQL),45,72,195,795,26,0,77,106,0,95,3,0,39,191,0,0
Lost,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
No show,0,0,3,30,0,0,1,3,0,1,0,0,5,4,0,0
Offer accepted,0,0,1,0,0,0,0,0,0,0,0,0,0,6,1,0
Opportunity,0,0,48,52,0,0,39,29,0,0,0,0,0,61,21,0
Proposal sent - Negotiation,0,0,21,7,0,0,3,3,0,0,0,0,0,17,8,0
SQL/Follow-up after demo,0,0,146,235,0,0,59,54,0,1,0,0,0,88,54,0
